In [70]:
from redis import Redis
from os import environ
redis_connection = Redis('this_redis')

# Redis

Redis is a key-value store. Values can be stored in redis associated with a given key.

In [69]:
redis_connection.set('foo', 42)

True

In [3]:
redis_connection.set('bar', [1,2,3])

True

In [4]:
redis_connection.get('bar')

b'[1, 2, 3]'

In [5]:
import requests
request = requests.get('http://this_api:5000/redis/bar')
print(request.text )

<h1>Your Value</h1><p>bar: [1, 2, 3]</p>


One interesting use case is to have Redis track the evolution of some long running iterative process.

In [6]:
import time

def some_iterative_process():
    time.sleep(1)

In [7]:
some_iterative_process()

In [81]:
count = 0
redis_connection.set('count', count)

True

In [82]:
while count < 30:
    some_iterative_process()
    count = redis_connection.incr('count')

In [83]:
count

30

## Passing Parameters Between Processes

We could also use Redis to pass parameters between procesess.

In [13]:
import numpy as np
import json
model_params = {'C':list(np.logspace(-3,3,7)),
                'penalty':'l1',
                'solver' : 'newton-cg'}

In [14]:
np.logspace(-3,3,7)

array([  1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
         1.00000000e+03])

In [20]:
list(np.logspace(-3,3,7))

[0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0]

In [24]:
model_params

{'C': [0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

In [25]:
type(model_params)

dict

In [26]:
json.dumps(model_params) #converts into a string. 

'{"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], "penalty": "l1", "solver": "newton-cg"}'

In [86]:
redis_connection.set('model_params', json.dumps(model_params))

True

In [87]:
type(redis_connection.set('model_params', json.dumps(model_params)))

bool

## Passing Arrays Between Processes

We could also use Redis to pass arrays between procesess.

In [38]:
A = np.array([[1,1,1],
              [2,2,2],
              [3,3,3]])

In [43]:
n, m = A.shape

In [45]:
n, m

(3, 3)

In [90]:
#json.dumps(A)
#not a serialization. Doesn't work.

In [93]:
A.ravel()

array([1, 1, 1, 2, 2, 2, 3, 3, 3])

In [47]:
encoded_A = A.ravel().tostring() #change to binary

In [49]:
encoded_A

b'\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00'

In [50]:
redis_connection.set('encoded_A', encoded_A)

True

In [51]:
redis_connection.set('A_n', n)
redis_connection.set('A_m', m)

True

In [52]:
def matrix_to_redis(matrix, redis_client, key):
    n, m = matrix.shape
    encoded_mat = matrix.ravel().tostring()
    redis_client.set(key, encoded_mat)
    redis_client.set(key+'_n', n)
    redis_client.set(key+'_m', m)

In [94]:
matrix_to_redis(A, redis_connection, 'new_A')

This could also be done via the `pickle` library.

In [95]:
import pickle
import numpy as np

B = np.random.randint(1,10,9).reshape(3,3)
B_pkl = pickle.dumps(B)

redis_connection.set('B', B_pkl)

True

In [96]:
B

array([[1, 3, 4],
       [3, 8, 2],
       [7, 4, 8]])

In [64]:
B_pkl

b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\x03K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i8q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89CH\x07\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.'

In [58]:
test = np.random.randint(1,10,9)
test

array([9, 5, 7, 2, 9, 1, 8, 4, 7])

In [60]:
test.reshape(3,3)

array([[9, 5, 7],
       [2, 9, 1],
       [8, 4, 7]])